In [40]:
%matplotlib inline 

import os
import json 
import time
import pickle
import pandas as pd 
import seaborn as sns

from sklearn.datasets.base import Bunch
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.exceptions import NotFittedError
from sklearn import metrics


from sklearn.model_selection import StratifiedKFold #ensure allows for randomization
from sklearn.model_selection import train_test_split as tts #drop this if using KFold
from sklearn import preprocessing
from sklearn.model_selection import validation_curve

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import LinearSVC, NuSVC, SVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression, SGDClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier

from yellowbrick.classifier import ClassificationReport
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

**Read in data, create the two binaries that I have to create outside the Pipeline**

In [19]:
filepath = os.path.abspath(os.path.join("/Users/tamananaheeme/Desktop/fixtures/hmda2013sample_balanced.csv"))
dataset = pd.read_csv(filepath, low_memory=False)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 24 columns):
tract_to_msamd_income             48110 non-null float64
population                        48115 non-null float64
minority_population               48115 non-null float64
number_of_owner_occupied_units    48104 non-null float64
number_of_1_to_4_family_units     48107 non-null float64
loan_amount_000s                  50000 non-null int64
hud_median_family_income          48116 non-null float64
applicant_income_000s             50000 non-null float64
property_type_name                50000 non-null object
owner_occupancy_name              50000 non-null object
loan_type_name                    50000 non-null object
loan_purpose_name                 50000 non-null object
lien_status_name                  50000 non-null object
hoepa_status_name                 50000 non-null object
co_applicant_sex_name             50000 non-null object
co_applicant_race_name_1          50000 non-n

In [4]:
dataset.describe(include="all")

,tract_to_msamd_income,population,minority_population,number_of_owner_occupied_units,number_of_1_to_4_family_units,loan_amount_000s,hud_median_family_income,applicant_income_000s,property_type_name,owner_occupancy_name,...,co_applicant_sex_name,co_applicant_race_name_1,co_applicant_ethnicity_name,as_of_year,applicant_sex_name,applicant_race_name_1,applicant_ethnicity_name,agency_abbr,locality_type,action_taken
count,48110.000000,48115.000000,48115.000000,48104.000000,48107.000000,50000.000000,48116.000000,50000.000000,50000,50000,...,50000,50000,50000,50000.0,50000,50000,50000,50000,50000.000000,50000.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,...,5,8,5,NaN,4,7,4,6,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,One-to-four family dwelling (other than manufa...,Owner-occupied as a principal dwelling,...,No co-applicant,No co-applicant,No co-applicant,NaN,Male,White,Not Hispanic or Latino,CFPB,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48195,44339,...,25535,25535,25535,NaN,29882,34088,36558,24309,NaN,NaN
mean,115.300162,5354.659025,31.294497,1420.942915,1868.019394,203.121120,67477.148973,7425.796340,NaN,NaN,...,NaN,NaN,NaN,2013.0,NaN,NaN,NaN,NaN,0.842020,0.500000
std,42.033747,2637.838567,26.257307,724.386928,902.680785,554.861797,14495.695316,20606.344736,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.364726,0.500005
min,6.350000,33.000000,0.550000,7.000000,6.000000,1.000000,16400.000000,1.000000,NaN,NaN,...,NaN,NaN,NaN,2013.0,NaN,NaN,NaN,NaN,0.000000,0.000000
25%,88.422499,3723.000000,10.470000,942.000000,1297.000000,90.000000,58500.000000,48.000000,NaN,NaN,...,NaN,NaN,NaN,2013.0,NaN,NaN,NaN,NaN,1.000000,0.000000
50%,109.075001,4965.000000,22.500000,1328.000000,1742.000000,150.000000,65400.000000,81.000000,NaN,NaN,...,NaN,NaN,NaN,2013.0,NaN,NaN,NaN,NaN,1.000000,0.500000
75%,134.617496,6395.000000,45.290001,1773.000000,2284.000000,244.000000,73400.000000,149.000000,NaN,NaN,...,NaN,NaN,NaN,2013.0,NaN,NaN,NaN,NaN,1.000000,1.000000


In [20]:

remove = ['purchaser_type_name',
        'preapproval_name',
        'rate_spread', 
        'sequence_number', 
        'respondent_id',
        'state_name',
        'state_abbr',
        'county_name',
        'edit_status_name', 
        'denial_reason_name_3', 
        'denial_reason_name_2', 
        'denial_reason_name_1', 
        'co_applicant_race_name_5', 
        'co_applicant_race_name_4', 
        'co_applicant_race_name_3', 
        'co_applicant_race_name_2',
        'census_tract_number',
        'application_date_indicator', 
        'applicant_race_name_5', 
        'applicant_race_name_4', 
        'applicant_race_name_3', 
        'applicant_race_name_2', 
        'agency_name',
        'action_taken_name', 
        'msamd_nm']

non_bool_numeric = ['tract_to_msamd_income', 
        'population', 
        'minority_population', 
        'number_of_owner_occupied_units', 
        'number_of_1_to_4_family_units', 
        'loan_amount_000s', 
        'hud_median_family_income', 
        'applicant_income_000s',
        ]

categorical = ['property_type_name', 
        'owner_occupancy_name', 
        'loan_type_name', 
        'loan_purpose_name',
        'lien_status_name', 
        'hoepa_status_name', 
        'co_applicant_sex_name',
        'co_applicant_race_name_1', 
        'co_applicant_ethnicity_name', 
        'as_of_year',
        'applicant_sex_name', 
        'applicant_race_name_1',
        'applicant_ethnicity_name', 
        'agency_abbr']


In [25]:
dataset.fillna(dataset.mean(), inplace=True)

In [26]:
X = dataset[[col for col in dataset.columns if col != 'action_taken']]
y = dataset['action_taken']

In [27]:
print(dataset.shape)
print(X.shape)
dataset.describe(include='all')

(50000, 24)
(50000, 23)


,tract_to_msamd_income,population,minority_population,number_of_owner_occupied_units,number_of_1_to_4_family_units,loan_amount_000s,hud_median_family_income,applicant_income_000s,property_type_name,owner_occupancy_name,...,co_applicant_sex_name,co_applicant_race_name_1,co_applicant_ethnicity_name,as_of_year,applicant_sex_name,applicant_race_name_1,applicant_ethnicity_name,agency_abbr,locality_type,action_taken
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000,50000,...,50000,50000,50000,50000.0,50000,50000,50000,50000,50000.000000,50000.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,...,5,8,5,NaN,4,7,4,6,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,One-to-four family dwelling (other than manufa...,Owner-occupied as a principal dwelling,...,No co-applicant,No co-applicant,No co-applicant,NaN,Male,White,Not Hispanic or Latino,CFPB,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48195,44339,...,25535,25535,25535,NaN,29882,34088,36558,24309,NaN,NaN
mean,115.300162,5354.659025,31.294497,1420.942915,1868.019394,203.121120,67477.148973,7425.796340,NaN,NaN,...,NaN,NaN,NaN,2013.0,NaN,NaN,NaN,NaN,0.842020,0.500000
std,41.231640,2587.636607,25.757592,710.519541,885.427817,554.861797,14219.968617,20606.344736,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.364726,0.500005
min,6.350000,33.000000,0.550000,7.000000,6.000000,1.000000,16400.000000,1.000000,NaN,NaN,...,NaN,NaN,NaN,2013.0,NaN,NaN,NaN,NaN,0.000000,0.000000
25%,89.410004,3775.000000,10.880000,957.000000,1315.000000,90.000000,58600.000000,48.000000,NaN,NaN,...,NaN,NaN,NaN,2013.0,NaN,NaN,NaN,NaN,1.000000,0.000000
50%,110.699997,5064.000000,23.865001,1358.000000,1779.000000,150.000000,66000.000000,81.000000,NaN,NaN,...,NaN,NaN,NaN,2013.0,NaN,NaN,NaN,NaN,1.000000,0.500000
75%,133.399994,6328.000000,44.020000,1751.000000,2258.000000,244.000000,73300.000000,149.000000,NaN,NaN,...,NaN,NaN,NaN,2013.0,NaN,NaN,NaN,NaN,1.000000,1.000000


From Dr. Bengfort's Census Notebook:

This code creates a `meta.json` file by inspecting the data frame that we have constructued. The `target_names` column, is just the two unique values in the `data.income` series; by using the `pd.Series.unique` method - we're guarenteed to spot data errors if there are more or less than two values. The `feature_names` is simply the names of all the columns. 

Now that we have everything we need stored on disk, we can create a `load_data` function, which will allow us to load the training and test datasets appropriately from disk and store them in a `Bunch`: 

In [30]:
dataset.to_csv("/Users/tamananaheeme/Desktop/fixtures/hmda2013sample_balanced_formatted.csv", index=False)

In [31]:
meta = {
    'target_names': [str(yi) for yi in y.unique()],
    'feature_names': list(X.columns),
    'categorical_features': {
        column: list(X[column].unique())
        for column in X.columns
        if X[column].dtype == 'object'
    },
}

with open('data/meta.json', 'w') as f:
     json.dump(meta, f, indent=2)

In [32]:
def load_data(root=os.getcwd()):
    # Construct the `Bunch` for the HMDA dataset
    filenames     = {
        'meta': os.path.join(root, 'data','meta.json'),
        'rdme': os.path.join(root, 'data', 'readme.md'),
        'data': os.path.abspath(os.path.join("/Users/tamananaheeme/Desktop/fixtures/hmda2013sample_balanced_formatted.csv")),
    }

    # Load the meta data from the meta json
    with open(filenames['meta'], 'r') as f:
        meta = json.load(f)
        target_names  = meta['target_names']
        feature_names = meta['feature_names']

    # Load the description from the README. 
    with open(filenames['rdme'], 'r') as f:
        DESCR = f.read()

    # Load the dataset from the text file.
    dataset = pd.read_csv(filenames['data'], low_memory=False)

    # Extract the target from the data
    data = dataset[[col for col in dataset.columns if col != 'action_taken']]
    target = dataset['action_taken']

    # Create the bunch object
    return Bunch(
        data=data,
        target=target,
        filenames=filenames,
        target_names=target_names,
        feature_names=feature_names,
        categorical_features = meta['categorical_features'], 
        DESCR=DESCR
    )

# Save the dataset as a variable we can use.
dataset = load_data()

print(dataset.data.shape)

(50000, 23)


From Dr. Bengfort's Census Notebook:

Now that we've finally acheived our feature extraction, we can continue on to the model build phase. To create our classifier, we're going to create a [`Pipeline`](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) that uses our feature transformers and ends in an estimator that can do classification. We can then write the entire pipeline object to disk with the `pickle`, allowing us to load it up and use it to make predictions in the future. 

A pipeline is a step-by-step set of transformers that takes input data and transforms it, until finally passing it to an estimator at the end. Pipelines can be constructed using a named declarative syntax so that they're easy to modify and develop. Our pipeline is as follows:

In [33]:
class IdentityTransformer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X

**Create Pipeline**

In [34]:
# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, non_bool_numeric),
        ('cat', categorical_transformer, categorical)])

In [12]:
preprocessor.fit_transform(dataset.data).shape

(50000, 67)

**Test Pipeline**

In [35]:
X = dataset.data
y = dataset.target

In [36]:
#Source: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
categorical_indexes = [
    list(dataset.data.columns).index(key) for key in dataset.categorical_features.keys()
]

tn_chart_preprocessor = ColumnTransformer(
    transformers=[('cat', categorical_transformer, categorical)])

tn_chart_pipeline = Pipeline(steps=[('preprocessor', clone(preprocessor))])
        
chart_enc = tn_chart_pipeline

X_CHARTS = chart_enc.fit_transform(dataset.data)
X_CHARTS.shape


(50000, 67)

In [41]:
def fit_and_evaluate(X, y, model, label, **kwargs):

    start  = time.time() # Start the clock! 
    #scores = {'precision':[], 'recall':[], 'accuracy':[], 'f1':[]}

    scores = {'precision':[], 'precision_1':[], 'precision_0':[], 'recall':[], 'recall_1':[], 'recall_0':[], 'accuracy':[], 'f1':[],  'f1_1':[],  'f1_0':[]}
    
    skf = StratifiedKFold(n_splits = 12, shuffle=True, random_state=1)
    
    for train, test in skf.split(X, y):
        X_train, X_test = X.iloc[train], X.iloc[test]
        y_train, y_test = y[train], y[test]
        
        estimator = Pipeline(steps=[('preprocessor', clone(preprocessor)),
                      ('model', model(**kwargs))])
        
        estimator.fit(X_train, y_train)
        
        expected  = y_test
        predicted = estimator.predict(X_test)
        cr = classification_report(y_test, predicted, output_dict=True)
        
        # Append our scores to the tracker
        scores['precision'].append(metrics.precision_score(expected, predicted, average="weighted"))
        scores['precision_1'].append(cr['1']['precision'])
        scores['precision_0'].append(cr['0']['precision'])
        scores['recall'].append(metrics.recall_score(expected, predicted, average="weighted"))
        scores['recall_1'].append(cr['1']['recall'])
        scores['recall_0'].append(cr['0']['recall'])
        scores['accuracy'].append(metrics.accuracy_score(expected, predicted))
        scores['f1'].append(metrics.f1_score(expected, predicted, average="weighted"))
        scores['f1_1'].append(cr['1']['f1-score'])
        scores['f1_0'].append(cr['0']['f1-score'])
        
    # Report
    print("Build and Validation of {} took {:0.3f} seconds".format(label, time.time()-start))
    print("Validation scores are as follows:\n")
    print(pd.DataFrame(scores).mean())
    
    # Write official estimator to disk
    estimator = Pipeline(steps=[('preprocessor', clone(preprocessor)),
                      ('model', model(**kwargs))])
    estimator.fit(X, y)
    
    outpath = label.lower().replace(" ", "-") + ".pickle"
    with open(outpath, 'wb') as f:
        pickle.dump(estimator, f)

    print("\nFitted model written to:\n{}".format(os.path.abspath(outpath)))

In [42]:
fit_and_evaluate(X, y, LogisticRegression, "HMDA Logistic Regression", max_iter=6000)

Build and Validation of HMDA Logistic Regression took 11.379 seconds
Validation scores are as follows:

precision      0.671752
precision_1    0.707092
precision_0    0.636412
recall         0.664460
recall_1       0.561600
recall_0       0.767320
accuracy       0.664460
f1             0.660861
f1_1           0.625974
f1_0           0.695747
dtype: float64

Fitted model written to:
/Users/tamananaheeme/Desktop/GU_DS/Loan-Canoe/foo/hmda-logistic-regression.pickle


In [43]:
fit_and_evaluate(X, y, RandomForestClassifier, "HMDA Random Forest Classifier", n_estimators=100)

Build and Validation of HMDA Random Forest Classifier took 95.716 seconds
Validation scores are as follows:

precision      0.685883
precision_1    0.709009
precision_0    0.662757
recall         0.682959
recall_1       0.620599
recall_0       0.745320
accuracy       0.682959
f1             0.681698
f1_1           0.661812
f1_0           0.701584
dtype: float64

Fitted model written to:
/Users/tamananaheeme/Desktop/GU_DS/Loan-Canoe/foo/hmda-random-forest-classifier.pickle


In [44]:
fit_and_evaluate(X, y, LinearDiscriminantAnalysis, "HMDA Linear Discriminant Analysis")

Build and Validation of HMDA Linear Discriminant Analysis took 7.777 seconds
Validation scores are as follows:

precision      0.672809
precision_1    0.712099
precision_0    0.633518
recall         0.663860
recall_1       0.550160
recall_0       0.777559
accuracy       0.663860
f1             0.659447
f1_1           0.620718
f1_0           0.698177
dtype: float64

Fitted model written to:
/Users/tamananaheeme/Desktop/GU_DS/Loan-Canoe/foo/hmda-linear-discriminant-analysis.pickle


In [45]:
fit_and_evaluate(X, y, LinearSVC, "HMDA LinearSVC")

Build and Validation of HMDA LinearSVC took 91.909 seconds
Validation scores are as follows:

precision      0.672186
precision_1    0.709082
precision_0    0.635289
recall         0.664260
recall_1       0.557120
recall_0       0.771400
accuracy       0.664260
f1             0.660350
f1_1           0.623953
f1_0           0.696746
dtype: float64

Fitted model written to:
/Users/tamananaheeme/Desktop/GU_DS/Loan-Canoe/foo/hmda-linearsvc.pickle


In [46]:
fit_and_evaluate(X, y, BernoulliNB, "HMDA BernoulliNB")

Build and Validation of HMDA BernoulliNB took 3.788 seconds
Validation scores are as follows:

precision      0.655840
precision_1    0.697257
precision_0    0.614423
recall         0.644800
recall_1       0.512001
recall_0       0.777600
accuracy       0.644800
f1             0.638404
f1_1           0.590382
f1_0           0.686425
dtype: float64

Fitted model written to:
/Users/tamananaheeme/Desktop/GU_DS/Loan-Canoe/foo/hmda-bernoullinb.pickle


In [47]:
fit_and_evaluate(X, y, GaussianNB, "HMDA GaussianNB")

Build and Validation of HMDA GaussianNB took 3.953 seconds
Validation scores are as follows:

precision      0.769772
precision_1    0.969578
precision_0    0.569967
recall         0.621760
recall_1       0.251440
recall_0       0.992080
accuracy       0.621760
f1             0.561604
f1_1           0.399224
f1_0           0.723984
dtype: float64

Fitted model written to:
/Users/tamananaheeme/Desktop/GU_DS/Loan-Canoe/foo/hmda-gaussiannb.pickle


In [48]:
fit_and_evaluate(X, y, tree.DecisionTreeClassifier, "HMDA DecisionTreeClassifier")

Build and Validation of HMDA DecisionTreeClassifier took 11.764 seconds
Validation scores are as follows:

precision      0.624199
precision_1    0.623032
precision_0    0.625367
recall         0.624159
recall_1       0.628999
recall_0       0.619319
accuracy       0.624159
f1             0.624126
f1_1           0.625964
f1_0           0.622289
dtype: float64

Fitted model written to:
/Users/tamananaheeme/Desktop/GU_DS/Loan-Canoe/foo/hmda-decisiontreeclassifier.pickle


In [49]:
fit_and_evaluate(X, y, BaggingClassifier, "HMDA BaggingClassifier")     

Build and Validation of HMDA BaggingClassifier took 56.323 seconds
Validation scores are as follows:

precision      0.667550
precision_1    0.697444
precision_0    0.637657
recall         0.662180
recall_1       0.572719
recall_0       0.751640
accuracy       0.662180
f1             0.659426
f1_1           0.628906
f1_0           0.689945
dtype: float64

Fitted model written to:
/Users/tamananaheeme/Desktop/GU_DS/Loan-Canoe/foo/hmda-baggingclassifier.pickle


In [50]:
fit_and_evaluate(X, y, ExtraTreesClassifier, "HMDA ExtraTreesClassifier", n_estimators=100)

Build and Validation of HMDA ExtraTreesClassifier took 78.540 seconds
Validation scores are as follows:

precision      0.666152
precision_1    0.680513
precision_0    0.651791
recall         0.664879
recall_1       0.621479
recall_0       0.708280
accuracy       0.664879
f1             0.664228
f1_1           0.649620
f1_0           0.678835
dtype: float64

Fitted model written to:
/Users/tamananaheeme/Desktop/GU_DS/Loan-Canoe/foo/hmda-extratreesclassifier.pickle


In [51]:
fit_and_evaluate(X, y, SGDClassifier, "HMDA SGDClassifier")

Build and Validation of HMDA SGDClassifier took 3.768 seconds
Validation scores are as follows:

precision      0.653693
precision_1    0.679850
precision_0    0.627535
recall         0.632101
recall_1       0.576354
recall_0       0.687849
accuracy       0.632101
f1             0.618256
f1_1           0.601452
f1_0           0.635060
dtype: float64

Fitted model written to:
/Users/tamananaheeme/Desktop/GU_DS/Loan-Canoe/foo/hmda-sgdclassifier.pickle


The last step is to save our model to disk for reuse later, with the `pickle` module:

In [4]:
#import pickle 

def dump_model(model, path='data', name='classifier.pickle'):
    with open(os.path.join(path, name), 'wb') as f:
        pickle.dump(model, f)
        
dump_model(census)